In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bs4 import BeautifulSoup
import pandas as pd 
import matplotlib.pyplot as plt
import requests
import time


## Boardgame Geek data

In [2]:
simple_data = pd.read_csv("simple_BGG.csv")

Do not want all the complex data eg what rank of military game this game is if it even is a military game
Realised that I needed to include the alternative names as well - as games such as Catan are also known as many other names. 


In [3]:
cols_to_use = ['primary','average','averageweight','boardgamemechanic', 'id', 'maxplayers','minplayers','maxplaytime',
               'minplaytime','minage', 'median', 'numcomments', 'numweights', 'owned', 'playingtime', 'stddev',
               'usersrated', 'yearpublished']

detailed_data = pd.read_csv("detailed_BGG.csv", usecols = cols_to_use) 

In [4]:
simple_data.head(1)

,ID,Name,Year,Rank,Average,Bayes average,Users rated,URL,Thumbnail
0,174430,Gloomhaven,2017,1,8.9,8.611,24574,/boardgame/174430/gloomhaven,https://cf.geekdo-images.com/micro/img/8JYMPXd...


In [5]:
detailed_data.head(1)


,average,averageweight,boardgamemechanic,id,maxplayers,maxplaytime,median,minage,minplayers,minplaytime,numcomments,numweights,owned,playingtime,primary,stddev,usersrated,yearpublished
0,7.19118,2.3399,"['Dice Rolling', 'Modular Board', 'Route/Netwo...",13,4,120,0,10,3,60,16330,6866,125254,120,Catan,1.47121,85332,1995


Need to work out a way to combine the datasets. This can either be done by name or id. Potential issues: Id might not match. Names might not match. Might need to cross check the names within the primary and alternative

In [6]:
len(simple_data)

17065

In [7]:
len(detailed_data)

17063

Checking if the names match up: 

In [8]:
simple_names = list(simple_data.Name)

In [9]:
detailed_names = list(detailed_data.primary)

In [10]:
match = 0 
no_match = 0
unmatched_games = []
for simple in simple_names:
    if simple in detailed_names:
        match += 1
        detailed_names.remove(simple)
    else: 
        unmatched_games.append(simple)
        no_match += 1
        

In [11]:
print("Percent of games whose names match across both datasets:")
print(match / len(simple_names))

Percent of games whose names match across both datasets:
0.9986522121300908


These are the games in the simple_data which didn't find a match

In [12]:
print(sorted(unmatched_games))

['"Oh My God! There\'s An Axe In My Head." The Game of International Diplomacy ', '"Tarleton\'s Quarter!"', 'A House Divided', 'Admiral Ackbar "It\'s a TRAP!" GAME', 'Bob Jiten', 'Catacombs (third edition)', 'Conflict of Heroes: Awakening the Bear! (second edition)', 'Evolution (2005)', 'Faerie Tales', 'Fantastiqa', 'Guerra en el Pacifico', 'Jok R Ummy', 'Logic Labyrinth', 'MBT (First Edition)', 'Nagaraja', 'Noch mal!', 'Nonsense', 'Nētā-Tanka', 'Organ ATTACK!', 'The Walking Dead "Don\'t Look Back" Dice Game', 'Tybor The Builder', 'Victory:  The Blocks of War', 'Wooly Bully']


The games in the detailed data which didn't find a match:

In [13]:
print(sorted(detailed_names))

['"Oh My God! There\\\'s An Axe In My Head." The Game of International Diplomacy ', '"Tarleton\\\'s Quarter!"', 'A House Divided: War Between the States 1861-65', 'Admiral Ackbar "It\\\'s a TRAP!" GAME', 'Catacombs (Third Edition)', 'Conflict of Heroes: Awakening the Bear! – Operation Barbarossa 1941 (second edition)', 'Draco Mundis', 'Encore!', 'Evolution', 'Fantastiqa: The Rucksack Edition', 'Guerra en el Pacífico', 'Jok-R-ummy', 'MBT (first edition)', 'Naga Raja', 'Nonsense Classic', 'Nētā-Tanka (Deluxe Edition)', 'The Walking Dead "Don\\\'t Look Back" Dice Game', 'Tybor the Builder', 'Victory: World War II', 'Wooly Wars', 'ボブジテン (Bob Jiten)']


It can be seen that the majority of these "unmatched" games simply just need some cleaning up so that their strings match correctly. 

In [14]:
def clean_bg_data(simple_name, detailed_name, new_name):
    simple_idx = simple_data.index[simple_data["Name"] == simple_name][0]
    detailed_idx = detailed_data.index[detailed_data["primary"] == detailed_name][0]
    print(simple_data.loc[simple_idx])
    print(" ")
    print(detailed_data.loc[detailed_idx])    

In [15]:
clean_bg_data("A House Divided", "A House Divided: War Between the States 1861-65", " ")

ID                                                             701
Name                                               A House Divided
Year                                                          1981
Rank                                                          1071
Average                                                       7.08
Bayes average                                                6.416
Users rated                                                   1817
URL                                   /boardgame/701/house-divided
Thumbnail        https://cf.geekdo-images.com/micro/img/T6ji_rc...
Name: 1070, dtype: object
 
average                                                      7.08038
averageweight                                                 2.6473
boardgamemechanic        ['Dice Rolling', 'Point to Point Movement']
id                                                               701
maxplayers                                                         2
maxplaytime             

Made me appreciate actually that the labels might match up. So I thought I would test that instead of manually changing all the slightly wrong names:

In [16]:
print(len(simple_data))
print(len(detailed_data))


17065
17063


In [17]:
for i in simple_data.ID[:10]:
    print(i)

174430
161936
182028
167791
12333
187645
220308
169786
193738
233078


In [18]:
match = 0 
no_match = 0
no_match_simple = []
no_match_detailed = []
no_id_match = []
for game_id in simple_data.ID:    
    
    simple_loc = simple_data.loc[simple_data["ID"] == game_id, "Name"]
    detailed_loc = detailed_data.loc[detailed_data["id"] == game_id, "primary"]
    if len(detailed_loc) == 0: 
        no_id_match.append(game_id)
    else:
           
        simple_name = simple_loc.iloc[0]
        detailed_name = detailed_loc.iloc[0]

        #print(simple_name, detailed_name)


        if simple_name == detailed_name:
            match += 1
        else: 
            no_match += 1
            no_match_simple.append(simple_name)
            no_match_detailed.append(detailed_name)
    

In [19]:
print(len(no_id_match))
print(match, no_match)

2
17042 21


In [20]:
for i in range(no_match):
    print(no_match_simple[i])
    print(no_match_detailed[i])
    print(" ")

Conflict of Heroes: Awakening the Bear! (second edition)
Conflict of Heroes: Awakening the Bear! – Operation Barbarossa 1941 (second edition)
 
Catacombs (third edition)
Catacombs (Third Edition)
 
A House Divided
A House Divided: War Between the States 1861-65
 
Noch mal!
Encore!
 
Fantastiqa
Fantastiqa: The Rucksack Edition
 
Tybor The Builder
Tybor the Builder
 
Wooly Bully
Wooly Wars
 
MBT (First Edition)
MBT (first edition)
 
Victory:  The Blocks of War
Victory: World War II
 
Nētā-Tanka
Nētā-Tanka (Deluxe Edition)
 
Nagaraja
Naga Raja
 
"Tarleton's Quarter!"
"Tarleton\'s Quarter!"
 
Faerie Tales
Draco Mundis
 
Nonsense
Nonsense Classic
 
Bob Jiten
ボブジテン (Bob Jiten)
 
Guerra en el Pacifico
Guerra en el Pacífico
 
Jok R Ummy
Jok-R-ummy
 
"Oh My God! There's An Axe In My Head." The Game of International Diplomacy 
"Oh My God! There\'s An Axe In My Head." The Game of International Diplomacy 
 
Evolution (2005)
Evolution
 
The Walking Dead "Don't Look Back" Dice Game
The Walking Dead 

This shows that even though the names might not match exactly, the id's do correspond to the correct games. Part of the confusion is due to games being given different names when published internationally. ie "Noch Mal!" literally translates to "Again!", leading to the english version of the game being called "Encore!".  

Overall, the simple dataset gives better naming, so that is the one I shall move ahead with

### Combining the datasets

Since the game id's translate correctly across datasets, I shall merge on that.

In [21]:
detailed_data = detailed_data.rename(columns = {"id" : "ID"})

In [22]:
complete_dataset = pd.merge(simple_data, detailed_data, on = "ID")

In [23]:
cols_to_delete = ["URL", "Thumbnail", "Average", "yearpublished", "primary", "Users rated", "median"]
complete_dataset = complete_dataset.drop(cols_to_delete, axis = 1)

In [24]:
complete_dataset.head()

,ID,Name,Year,Rank,Bayes average,average,averageweight,boardgamemechanic,maxplayers,maxplaytime,minage,minplayers,minplaytime,numcomments,numweights,owned,playingtime,stddev,usersrated
0,174430,Gloomhaven,2017,1,8.611,8.89906,3.7930,"['Campaign / Battle Card Driven', 'Cooperative...",4,120,12,1,60,4995,1082,41470,120,1.60270,25289
1,161936,Pandemic Legacy: Season 1,2015,2,8.489,8.64165,2.8284,"['Action Point Allowance System', 'Cooperative...",4,60,13,2,60,4984,874,49878,60,1.60713,31326
2,182028,Through the Ages: A New Story of Civilization,2015,3,8.258,8.53104,4.3714,"['Action Point Allowance System', 'Auction/Bid...",4,120,14,2,120,2309,770,19551,120,1.49173,15835
3,167791,Terraforming Mars,2016,4,8.242,8.40331,3.2397,"['Card Drafting', 'Hand Management', 'Set Coll...",5,120,12,1,120,5932,1452,50512,120,1.36045,39327
4,12333,Twilight Struggle,2005,5,8.175,8.32253,3.5660,"['Area Control / Area Influence', 'Campaign / ...",2,180,13,2,120,7757,3251,47495,180,1.58575,34535


Board mechanics is the least clean and most problematic measurement in this dataset, as each boardgame can come under different categories. 
We'll find clean up the formatting and find out how may unique categories of boardgames are listed here.

In [25]:
mechanics = []
for mech in complete_dataset.boardgamemechanic:
    if type(mech) == str:
        seperated_mech = mech.split(",")
        for i in seperated_mech:
            cleaned_mech = i.strip(' \'[]') 

            if cleaned_mech not in mechanics:
                mechanics.append(cleaned_mech)


In [26]:
print(len(mechanics))

53


In [27]:
print(mechanics)

['Campaign / Battle Card Driven', 'Cooperative Play', 'Grid Movement', 'Hand Management', 'Modular Board', 'Role Playing', 'Simultaneous Action Selection', 'Storytelling', 'Variable Player Powers', 'Action Point Allowance System', 'Point to Point Movement', 'Set Collection', 'Trading', 'Auction/Bidding', 'Card Drafting', 'Take That', 'Tile Placement', 'Area Control / Area Influence', 'Dice Rolling', 'Area Movement', 'Partnerships', 'Route/Network Building', 'Deck / Pool Building', 'Area-Impulse', 'Variable Phase Order', 'Voting', 'Action / Movement Programming', 'Worker Placement', 'Area Enclosure', 'Pattern Building', 'Press Your Luck', 'Memory', 'Pick-up and Deliver', 'Player Elimination', 'Commodity Speculation', 'Secret Unit Deployment', 'Time Track', 'Stock Holding', 'Hidden Traitor', 'Rondel', 'Hex-and-Counter', 'Line Drawing', 'Simulation', 'Betting/Wagering', 'Trick-taking', 'Roll / Spin and Move', 'Pattern Recognition', 'Paper-and-Pencil', 'Rock-Paper-Scissors', 'Acting', 'Cra

Need to think about how I will encode this data for the machine learning model - 53 new measurements is likely to significantly slow down the learning. See more about encoding here: https://towardsdatascience.com/all-about-categorical-variable-encoding-305f3361fd02

## Getting the prices for the games: 

In [28]:
game_name = "Agricola"
price_url = "https://boardgameprices.co.uk/api/search?search={}&currency=GBP&destination=GB&delivery=PACKAGE,POSTOFFICE&sort=SMART&sitename=boardgameprices.co.uk".format(game_name)
price_site = requests.get(price_url)
price_data = price_site.json()
top_result = price_data["items"][0]
top_price = top_result["bestprice"]

In [29]:
characters_needed = []
for name in complete_dataset.Name:
    for letter in name: 
        if letter not in characters_needed:
            characters_needed.append(letter)

In [30]:
print(sorted(characters_needed))

[' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '¥', '°', '²', '³', '·', '¿', 'Á', 'Â', 'Ä', 'Æ', 'È', 'É', 'Í', 'Ó', 'Ö', '×', 'Ú', 'Ü', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ì', 'í', 'ï', 'ð', 'ñ', 'ó', 'ô', 'õ', 'ö', 'ù', 'ú', 'û', 'ü', 'ā', 'ą', 'ć', 'Č', 'ē', 'ę', 'ě', 'İ', 'Ł', 'ł', 'ń', 'ō', 'ŏ', 'Ś', 'ś', 'Ŝ', 'Ū', 'ū', 'Ż', 'ż', 'ȗ', '́', 'С', 'Э', 'в', 'д', 'и', 'н', 'р', 'с', 'т', 'у', '‐', '–', '‘', '•', '…', '₂', '€', '∞', 'く', 'と', 'ひ', 'ふ', 'ら', 'ろ', 'イ', 'キ', 'ク', 'ス', 'ダ', 'ッ', 'テ', 'ト', 'プ', 'メ', 'ャ', 'ロ', 'ン', '则', '刺', '占', '団', '客', '小', '師', '庭', '扇', '拜', '敬', '服', '杀',

In [31]:
word = ""
for i in sorted(characters_needed):
    word += i

In [32]:
print(word)

 !"#$&'()*+,-./0123456789:?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz¡¥°²³·¿ÁÂÄÆÈÉÍÓÖ×ÚÜÞßàáâãäåæçèéêìíïðñóôõöùúûüāąćČēęěİŁłńōŏŚśŜŪūŻżȗ́СЭвдинрсту‐–‘•…₂€∞くとひふらろイキクスダッテトプメャロン则刺占団客小師庭扇拜敬服杀槌法猿老谋貨道金：


In [33]:
import urllib.parse

In [34]:
safe_items = '-.\"*0123456789<>abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [35]:
urllib.parse.quote_plus(word, safe = safe_items)

'+%21"%23%24%26%27%28%29*%2B%2C-.%2F0123456789%3A%3F%40ABCDEFGHIJKLMNOPQRSTUVWXYZ%5B%5D_abcdefghijklmnopqrstuvwxyz%C2%A1%C2%A5%C2%B0%C2%B2%C2%B3%C2%B7%C2%BF%C3%81%C3%82%C3%84%C3%86%C3%88%C3%89%C3%8D%C3%93%C3%96%C3%97%C3%9A%C3%9C%C3%9E%C3%9F%C3%A0%C3%A1%C3%A2%C3%A3%C3%A4%C3%A5%C3%A6%C3%A7%C3%A8%C3%A9%C3%AA%C3%AC%C3%AD%C3%AF%C3%B0%C3%B1%C3%B3%C3%B4%C3%B5%C3%B6%C3%B9%C3%BA%C3%BB%C3%BC%C4%81%C4%85%C4%87%C4%8C%C4%93%C4%99%C4%9B%C4%B0%C5%81%C5%82%C5%84%C5%8D%C5%8F%C5%9A%C5%9B%C5%9C%C5%AA%C5%AB%C5%BB%C5%BC%C8%97%CC%81%D0%A1%D0%AD%D0%B2%D0%B4%D0%B8%D0%BD%D1%80%D1%81%D1%82%D1%83%E2%80%90%E2%80%93%E2%80%98%E2%80%A2%E2%80%A6%E2%82%82%E2%82%AC%E2%88%9E%E3%81%8F%E3%81%A8%E3%81%B2%E3%81%B5%E3%82%89%E3%82%8D%E3%82%A4%E3%82%AD%E3%82%AF%E3%82%B9%E3%83%80%E3%83%83%E3%83%86%E3%83%88%E3%83%97%E3%83%A1%E3%83%A3%E3%83%AD%E3%83%B3%E5%88%99%E5%88%BA%E5%8D%A0%E5%9B%A3%E5%AE%A2%E5%B0%8F%E5%B8%AB%E5%BA%AD%E6%89%87%E6%8B%9C%E6%95%AC%E6%9C%8D%E6%9D%80%E6%A7%8C%E6%B3%95%E7%8C%BF%E8%80%81%E8%B0%8B%E8%B2%A8%E9%81%93%

In [36]:
def search_price(name):
    url_name = urllib.parse.quote_plus(name, safe = safe_items)
    price_url = "https://boardgameprices.co.uk/api/search?search={}&currency=GBP&destination=GB&delivery=PACKAGE,POSTOFFICE&sort=SMART&sitename=boardgameprices.co.uk".format(url_name)
    
    price_site = requests.get(price_url)
    
    price_data = price_site.json()
    
    if len(price_data["items"]) == 0:
        print("Error: No price found for {}".format(name))
        return -1
    else:
        
        top_result = price_data["items"][0]
        try:
            top_price = top_result["bestprice"]
        except: 
            top_price = -2 

        return top_price

In [37]:
search_price("too many bones")

-2

In [38]:
complete_dataset["price"] = 0.0

In [39]:
#updated_dataset = complete_dataset.copy()

In [40]:
#updated_dataset = updated_dataset.astype({"price" : float})

In [41]:
#updated_dataset.to_excel("updated_df.xlsx")

In [42]:
updated_dataset = pd.read_excel("updated_df.xlsx")

In [43]:
updated_dataset.head(2)

,ID,Name,Year,Rank,Bayes average,average,averageweight,boardgamemechanic,maxplayers,maxplaytime,minage,minplayers,minplaytime,numcomments,numweights,owned,playingtime,stddev,usersrated,price
0,174430,Gloomhaven,2017,1,8.611,8.89906,3.7930,"['Campaign / Battle Card Driven', 'Cooperative...",4,120,12,1,60,4995,1082,41470,120,1.60270,25289,88.95
1,161936,Pandemic Legacy: Season 1,2015,2,8.489,8.64165,2.8284,"['Action Point Allowance System', 'Cooperative...",4,60,13,2,60,4984,874,49878,60,1.60713,31326,51.73


In [44]:
hours = 4
minutes = 0
time_to_run = 60 * minutes + 3600 * hours
time_gap = 12
count = 1
max_count = 300

time_initial = time.time()

for game_id in updated_dataset.ID:
    
    time_count = time.time()
    running_duration = time_count - time_initial
    if running_duration < time_to_run:
    
        current_price = updated_dataset.loc[updated_dataset["ID"] == game_id, "price"].iloc[0]
        if current_price == 0: # Check to see that the game does not already have a price. 
            start = time.time()
            idx = updated_dataset[updated_dataset["ID"] == game_id].index.values[0]
            name = updated_dataset.loc[idx, "Name"]
            price = search_price(name)
            updated_dataset.at[idx, "price"] = price
            total_progress = updated_dataset[updated_dataset.price != 0].Name.count()
            count += 1
            end = time.time()
            duration = round(end - start, 3)
            print("{} - {}/{} - £{} - {}s - {}".format(total_progress, count, max_count, price, duration, name))

            time.sleep(max(0, (time_gap + np.random.random(1)[0]) - duration))    

        if count % 50 == 0: 
            print("\nStarting file update...")
            start = time.time()
            updated_dataset.to_excel("updated_df.xlsx")
            end = time.time()
            print("Updated excel file in {}s.\n".format(round(end - start, 2)))
            count += 1
    else:
        break

print("\nStarting file update...")
start = time.time()
updated_dataset.to_excel("updated_df.xlsx")
end = time.time()
print("Updated excel file in {}s.\n".format(round(end - start, 2)))


13541 - 2/300 - £35.65 - 0.21s - Aruba: Battle Race
13542 - 3/300 - £29.49 - 0.302s - Monopoly: Fallout Collector's Edition
13543 - 4/300 - £-2 - 0.186s - Dare to Dream
13544 - 5/300 - £14.95 - 0.215s - Okinawa
Error: No price found for Wissens-Spektrum
13545 - 6/300 - £-1 - 0.15s - Wissens-Spektrum
13546 - 7/300 - £30.95 - 0.251s - King Arthur
13547 - 8/300 - £19.16 - 0.199s - De 7 geitjes
Error: No price found for Quizzard
13548 - 9/300 - £-1 - 0.206s - Quizzard
Error: No price found for Scrabble Twists & Turns
13549 - 10/300 - £-1 - 0.166s - Scrabble Twists & Turns
13550 - 11/300 - £23.45 - 0.223s - MasterIslands
13551 - 12/300 - £-2 - 0.187s - Star Wars: Rule the Galaxy
13552 - 13/300 - £7.64 - 0.254s - Grimslingers: Duels
13553 - 14/300 - £27.92 - 0.693s - Fortuna
13554 - 15/300 - £12.19 - 0.255s - Interpol
13555 - 16/300 - £14.79 - 0.534s - Das Ligretto Fußballspiel
13556 - 17/300 - £7.95 - 0.18s - Race to the Meuse
Error: No price found for Songburst: 70's & 80's Edition
13557 -

Error: No price found for Rollout: The Game of the Risk-Takers
13664 - 127/300 - £-1 - 0.18s - Rollout: The Game of the Risk-Takers
13665 - 128/300 - £10.9 - 0.192s - Bücherwurm
13666 - 129/300 - £15.67 - 0.247s - Babylon
13667 - 130/300 - £-2 - 0.237s - A Dog's Life
Error: No price found for Octrix
13668 - 131/300 - £-1 - 0.183s - Octrix
Error: No price found for Movie Buff
13669 - 132/300 - £-1 - 0.186s - Movie Buff
13670 - 133/300 - £14.37 - 0.423s - Chaos
Error: No price found for The McDonald's Game
13671 - 134/300 - £-1 - 0.172s - The McDonald's Game
13672 - 135/300 - £13.49 - 0.236s - Too Many Cooks
13673 - 136/300 - £23.7 - 0.228s - Asia Crossroads
Error: No price found for Little Dungeon: Turtle Rock
13674 - 137/300 - £-1 - 0.18s - Little Dungeon: Turtle Rock
Error: No price found for Wie Hund und Katz!
13675 - 138/300 - £-1 - 0.177s - Wie Hund und Katz!
13676 - 139/300 - £-2 - 0.24s - Programmer's Nightmare
13677 - 140/300 - £-2 - 0.397s - Pitch Fleet
13678 - 141/300 - £24.45

13791 - 257/300 - £28.68 - 0.248s - Expeditie Robinson
Error: No price found for Tunebaya
13792 - 258/300 - £-1 - 0.191s - Tunebaya
13793 - 259/300 - £28.33 - 0.209s - Bakong
Error: No price found for Contigo
13794 - 260/300 - £-1 - 0.196s - Contigo
Error: No price found for Jorden runt på 80 dagar
13795 - 261/300 - £-1 - 0.181s - Jorden runt på 80 dagar
13796 - 262/300 - £18.28 - 0.233s - Viewpoint
13797 - 263/300 - £18 - 0.521s - CHEW: Cases of the FDA
Error: No price found for CrackeD ICE
13798 - 264/300 - £-1 - 0.183s - CrackeD ICE
13799 - 265/300 - £22.79 - 0.223s - Constellation
13800 - 266/300 - £23.51 - 0.238s - Electronic Dream Phone
Error: No price found for Penguin Soccer
13801 - 267/300 - £-1 - 0.19s - Penguin Soccer
13802 - 268/300 - £30.38 - 0.229s - Yummy Yummy Pancake
13803 - 269/300 - £13.19 - 0.212s - Fruit Salad
Error: No price found for Tangram Master
13804 - 270/300 - £-1 - 0.192s - Tangram Master
13805 - 271/300 - £91.95 - 0.504s - Goal
Error: No price found for S

13914 - 382/300 - £26.7 - 0.385s - The Italian Campaign: Salerno
13915 - 383/300 - £-2 - 0.216s - Putsch
13916 - 384/300 - £-2 - 0.222s - Fantasy Business
Error: No price found for Bag-O-Loot
13917 - 385/300 - £-1 - 0.203s - Bag-O-Loot
Error: No price found for Autobridge
13918 - 386/300 - £-1 - 0.174s - Autobridge
13919 - 387/300 - £11.32 - 0.221s - Just4Fun Colours
Error: No price found for Pro Golf
13920 - 388/300 - £-1 - 0.188s - Pro Golf
13921 - 389/300 - £-2 - 0.227s - Cock Block
Error: No price found for 007 James Bond: The Man with the Golden Gun
13922 - 390/300 - £-1 - 0.212s - 007 James Bond: The Man with the Golden Gun
Error: No price found for Scene It? 80s
13923 - 391/300 - £-1 - 0.209s - Scene It? 80s
Error: No price found for Prime Wars
13924 - 392/300 - £-1 - 0.407s - Prime Wars
Error: No price found for Rails Through the Rockies
13925 - 393/300 - £-1 - 0.207s - Rails Through the Rockies
Error: No price found for Babuschka
13926 - 394/300 - £-1 - 0.203s - Babuschka
Erro

14033 - 504/300 - £28.55 - 0.209s - Urban Panic
Error: No price found for Clontarf
14034 - 505/300 - £-1 - 0.192s - Clontarf
14035 - 506/300 - £25.25 - 0.35s - Monopoly: Nightmare Before Christmas Collector's Edition
14036 - 507/300 - £60.82 - 0.267s - Robin Hood
Error: No price found for LEGO Time Cruisers Game
14037 - 508/300 - £-1 - 0.226s - LEGO Time Cruisers Game
14038 - 509/300 - £59.19 - 20.204s - Clockwork Kingdom
14039 - 510/300 - £28.92 - 0.228s - Chicken Chase
14040 - 511/300 - £10.45 - 0.178s - Crisis: 2000
Error: No price found for Stockers!
14041 - 512/300 - £-1 - 0.181s - Stockers!
14042 - 513/300 - £24.98 - 3.052s - Bookmaker
Error: No price found for Tank Commander: The Eastern Front
14043 - 514/300 - £-1 - 0.219s - Tank Commander: The Eastern Front
Error: No price found for Abbeville 1940
14044 - 515/300 - £-1 - 0.169s - Abbeville 1940
14045 - 516/300 - £25.11 - 0.252s - Family Alias
Error: No price found for Star Reporter
14046 - 517/300 - £-1 - 0.208s - Star Reporte

14143 - 616/300 - £24.31 - 0.317s - Spirits!
Error: No price found for Scene It? Harry Potter Second Edition
14144 - 617/300 - £-1 - 0.162s - Scene It? Harry Potter Second Edition
14145 - 618/300 - £26.75 - 0.22s - Luther: Das Spiel
14146 - 619/300 - £-2 - 0.29s - Jolly Roger
Error: No price found for Los Viajes del Capitán Foucault
14147 - 620/300 - £-1 - 0.169s - Los Viajes del Capitán Foucault
14148 - 621/300 - £36.6 - 0.478s - Freche Frösche
14149 - 622/300 - £13.05 - 0.269s - Ligretto 2
14150 - 623/300 - £13.15 - 0.219s - Lord of the Dead
14151 - 624/300 - £46.11 - 0.287s - Good Help
14152 - 625/300 - £20.95 - 0.202s - The Italian Campaign: Anzio
14153 - 626/300 - £18.98 - 0.206s - Star Wars: Bounty Hunter – Das Würfelspiel
14154 - 627/300 - £-2 - 0.203s - Claim to Fame
14155 - 628/300 - £6.85 - 0.275s - No Game Over
Error: No price found for Froop!
14156 - 629/300 - £-1 - 0.185s - Froop!
14157 - 630/300 - £18 - 0.192s - Khrysos Hunters
Error: No price found for Les Cabanes de M's

14256 - 731/300 - £17.99 - 0.272s - Heart of Africa
14257 - 732/300 - £30.45 - 0.213s - War in the Aegean
Error: No price found for Mad Dash!
14258 - 733/300 - £-1 - 0.179s - Mad Dash!
14259 - 734/300 - £-2 - 0.204s - El Luchador Fantastico Grande
14260 - 735/300 - £34.86 - 0.205s - Narnia Risk Junior
Error: No price found for Witty Pong
14261 - 736/300 - £-1 - 0.17s - Witty Pong
14262 - 737/300 - £-2 - 0.195s - Cthulhu Mash
14263 - 738/300 - £32.14 - 0.387s - Super 3
14264 - 739/300 - £24.32 - 0.243s - Junior Stratego
14265 - 740/300 - £24.99 - 0.245s - Monopoly: Game of Thrones Collector's Edition
14266 - 741/300 - £28.95 - 0.205s - The Forever War
14267 - 742/300 - £15.59 - 0.196s - Chupacabra: Survive the Night
14268 - 743/300 - £23.56 - 0.214s - Kakuro Challenge
14269 - 744/300 - £-2 - 0.217s - Race to Riches
14270 - 745/300 - £11.98 - 0.211s - Trick-Taking: The Trick-Taking Game
14271 - 746/300 - £31.99 - 0.225s - Jim Henson's The Dark Crystal: Board Game
14272 - 747/300 - £9.04 

14379 - 857/300 - £10.45 - 0.228s - Stadt - Land - Fluss
14380 - 858/300 - £6.98 - 0.497s - Captain Treasure Boots
14381 - 859/300 - £21.74 - 0.215s - Paparazzi
14382 - 860/300 - £14.8 - 0.223s - Gierki Małżeńskie
Error: No price found for PsychoPet: The Nutty Pet Clinic
14383 - 861/300 - £-1 - 0.18s - PsychoPet: The Nutty Pet Clinic
Error: No price found for Creature Clash! Card Game
14384 - 862/300 - £-1 - 0.2s - Creature Clash! Card Game
14385 - 863/300 - £24.39 - 0.209s - Hagoth: Builder of Ships
14386 - 864/300 - £-2 - 0.237s - Money Lisa
14387 - 865/300 - £28.67 - 0.256s - Ex & Hopp
14388 - 866/300 - £28.71 - 0.265s - Chop Chop
14389 - 867/300 - £20.98 - 0.22s - Nitro Dice
14390 - 868/300 - £49.95 - 0.347s - Exploration
14391 - 869/300 - £23.52 - 0.183s - Vampire Werewolf Fairies
14392 - 870/300 - £30.94 - 0.285s - Treasure Hunt
14393 - 871/300 - £31.36 - 0.218s - Fleeced!
14394 - 872/300 - £21.71 - 0.232s - Bam!: Das unanständig gute Wortspiel
14395 - 873/300 - £-2 - 0.258s - Ti

14507 - 987/300 - £44.95 - 0.234s - Pipeline
14508 - 988/300 - £-2 - 0.195s - Die Kette von Saba
14509 - 989/300 - £-2 - 0.218s - Feed the Woozle
Error: No price found for Linkity
14510 - 990/300 - £-1 - 0.187s - Linkity
14511 - 991/300 - £34.77 - 0.229s - Redneck Life
14512 - 992/300 - £19.98 - 0.521s - Pittsburgh 68
14513 - 993/300 - £12.29 - 0.214s - Fruit Ninja Card Game
14514 - 994/300 - £-2 - 0.223s - Wreckage
14515 - 995/300 - £19.99 - 0.195s - United Nations
14516 - 996/300 - £26.03 - 0.492s - Monopoly: The Beatles Collector's Edition
14517 - 997/300 - £-2 - 0.219s - Schraumen
14518 - 998/300 - £-2 - 0.187s - G.O.O.T.M.U.
14519 - 999/300 - £-2 - 0.225s - Star Wars: Galaxy Rebellion
Error: No price found for Pirates of the Caribbean Buccaneer
14520 - 1000/300 - £-1 - 0.158s - Pirates of the Caribbean Buccaneer

Starting file update...
Updated excel file in 15.09s.

Error: No price found for Warchon
14521 - 1002/300 - £-1 - 0.176s - Warchon
Error: No price found for High Stakes D

14634 - 1117/300 - £9.95 - 0.217s - China Incident
14635 - 1118/300 - £-2 - 0.173s - Indiana Jones Akator Temple Race Game
14636 - 1119/300 - £-2 - 0.184s - Destination Hogwarts
14637 - 1120/300 - £32.78 - 0.213s - The Arabian Pots
14638 - 1121/300 - £-2 - 0.209s - Bloody Legacy
Error: No price found for Warhammer Warriors Battle Books
14639 - 1122/300 - £-1 - 0.197s - Warhammer Warriors Battle Books
14640 - 1123/300 - £28.76 - 0.263s - Speculation
14641 - 1124/300 - £-2 - 0.214s - Dudab
Error: No price found for Wildcraft!
14642 - 1125/300 - £-1 - 0.188s - Wildcraft!
14643 - 1126/300 - £11.99 - 0.211s - Dynamite
14644 - 1127/300 - £69.99 - 0.234s - Supremacy
Error: No price found for Guesstimation
14645 - 1128/300 - £-1 - 0.259s - Guesstimation
14646 - 1129/300 - £-2 - 0.214s - Cut and Run
Error: No price found for Wanna Bet?
14647 - 1130/300 - £-1 - 0.172s - Wanna Bet?
14648 - 1131/300 - £15.6 - 0.248s - Pass the Pigs: Pig Party Edition
14649 - 1132/300 - £12.59 - 0.565s - Pergamemno

In [223]:
updated_dataset.to_excel("updated_df.xlsx")


ConnectionError: HTTPSConnectionPool(host='boardgameprices.co.uk', port=443): Max retries exceeded with url: /api/search?search=Airborne+Commander&currency=GBP&destination=GB&delivery=PACKAGE,POSTOFFICE&sort=SMART&sitename=boardgameprices.co.uk (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000024E32402AC8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

In [133]:
for p in updated_dataset.price[:10]:
    print(p)

88.95
51.73
36.95
46.99
37.5
62.5
66.27
47.5
31.95
91.0


In [142]:
updated_dataset.head(25)

,ID,Name,Year,Rank,Bayes average,average,averageweight,boardgamemechanic,maxplayers,maxplaytime,minage,minplayers,minplaytime,numcomments,numweights,owned,playingtime,stddev,usersrated,price
0,174430,Gloomhaven,2017,1,8.611,8.89906,3.7930,"['Campaign / Battle Card Driven', 'Cooperative...",4,120,12,1,60,4995,1082,41470,120,1.60270,25289,88.95
1,161936,Pandemic Legacy: Season 1,2015,2,8.489,8.64165,2.8284,"['Action Point Allowance System', 'Cooperative...",4,60,13,2,60,4984,874,49878,60,1.60713,31326,51.73
2,182028,Through the Ages: A New Story of Civilization,2015,3,8.258,8.53104,4.3714,"['Action Point Allowance System', 'Auction/Bid...",4,120,14,2,120,2309,770,19551,120,1.49173,15835,36.95
3,167791,Terraforming Mars,2016,4,8.242,8.40331,3.2397,"['Card Drafting', 'Hand Management', 'Set Coll...",5,120,12,1,120,5932,1452,50512,120,1.36045,39327,46.99
4,12333,Twilight Struggle,2005,5,8.175,8.32253,3.5660,"['Area Control / Area Influence', 'Campaign / ...",2,180,13,2,120,7757,3251,47495,180,1.58575,34535,37.50
5,187645,Star Wars: Rebellion,2016,6,8.164,8.44551,3.6888,"['Area Control / Area Influence', 'Area Moveme...",4,240,14,2,180,2704,556,25463,240,1.37475,16732,62.50
6,220308,Gaia Project,2017,7,8.121,8.53656,4.2783,"['Area Control / Area Influence', 'Modular Boa...",4,150,12,1,60,1417,618,11575,150,1.43460,9442,66.27
7,169786,Scythe,2016,8,8.121,8.28410,3.3708,"['Area Control / Area Influence', 'Grid Moveme...",5,115,14,1,90,6281,1513,51977,115,1.42913,39962,47.50
8,193738,Great Western Trail,2016,9,8.082,8.28848,3.6844,"['Deck / Pool Building', 'Hand Management', 'P...",4,150,12,2,75,2636,751,23765,150,1.25429,19431,31.95
9,233078,Twilight Imperium (Fourth Edition),2017,10,8.076,8.72044,4.1891,"['Area Control / Area Influence', 'Area-Impuls...",6,480,14,3,240,1133,312,9526,480,1.58244,6851,91.00


In [125]:
dataTypeSeries = updated_dataset.dtypes
 
print('Data type of each column of Dataframe :')
print(dataTypeSeries)

Data type of each column of Dataframe :
ID                     int64
Name                  object
Year                   int64
Rank                   int64
Bayes average        float64
average              float64
averageweight        float64
boardgamemechanic     object
maxplayers             int64
maxplaytime            int64
minage                 int64
minplayers             int64
minplaytime            int64
numcomments            int64
numweights             int64
owned                  int64
playingtime            int64
stddev               float64
usersrated             int64
price                  int64
dtype: object


In [42]:
print(complete_dataset[complete_dataset["Name"] == "Terraforming Mars"].index.values)

[3]
